# Convert Exon Amino Acid Positions to Coding Sequence Domains (CDS)
## Resitance Mutations are show in final products, but databases store Amino Acid Coding Regions

We now need to translate.

Let's try to dio this with pyranges. – it seems to be made for this. 

Using this GFF file:
- https://github.com/LaraFuhrmann/Scan-for-mutations-of-interest-NGS-samples/blob/main/resources/GCF_009858895.2_ASM985889v3_genomic.gbff


In [7]:
# load from files

options = {
    "3C-like proteinase": '3CLpro_inhibitors_datasheet.csv',
    "RNA-dependent RNA polymerase": 'RdRP_inhibitors_datasheet.csv',
    "spike glycoprotein": 'spike_mAbs_datasheet.csv'
}

# load the data, the first time i each column is the amino acid change e.g. T124I, ignore the rest
dfs = {}
for product, file in options.items():
    try:
        # Read the CSV file
        df = pd.read_csv(file)
        # Store the dataframe in the dictionary with product as key
        dfs[product] = df
        print(f"Loaded {len(df)} mutations for {product}")
    except FileNotFoundError:
        print(f"Warning: File {file} not found")
    except Exception as e:
        print(f"Error loading {file}: {e}")

Loaded 71 mutations for 3C-like proteinase
Loaded 19 mutations for RNA-dependent RNA polymerase
Loaded 164 mutations for spike glycoprotein


In [12]:
import re
import pandas as pd

# Configuration for subregions
SUBREGIONS = {
    "RdRp": {
        "regions": [
            {"orf": "ORF1a", "start": 13442, "end": 13468, "parent_start": 266, "aa_range": (1, 9)},
            {"orf": "ORF1b", "start": 13468, "end": 16236, "parent_start": 13468, "aa_range": (10, 932)}
        ]
    },
    "3CLpro": {
        "regions": [
            {"orf": "ORF1a", "start": 10055, "end": 10972, "parent_start": 266, "aa_range": (1, 306)}
        ]
    }
}

def translate_mutation(mutation, orf, offset):
    """Translate a mutation by applying an offset and ORF prefix."""
    match = re.match(r'([A-Za-z])(\d+)([A-Za-z]|del)', mutation)
    if match:
        orig, pos, new = match.groups()
        new_pos = int(pos) + offset
        return f"{orf}:{orig}{new_pos}{new}"
    return None

def get_offset(subregion, position):
    """Determine the correct ORF and offset for a mutation position."""
    config = SUBREGIONS[subregion]
    for region in config["regions"]:
        start_aa, end_aa = region["aa_range"]
        if start_aa <= position <= end_aa:
            # Calculate offset: parent ORF amino acid start - subregion start
            parent_aa_start = ((region["start"] - region["parent_start"]) // 3) + 1
            offset = parent_aa_start - start_aa
            return region["orf"], offset
    raise ValueError(f"Position {position} out of range for {subregion}")

def translate_mutations(mutations, subregion):
    """Translate mutations to their parent ORFs."""
    if subregion not in SUBREGIONS:
        raise ValueError(f"Unknown subregion: {subregion}")
    
    translated = []
    for mutation in mutations:
        match = re.match(r'([A-Za-z])(\d+)([A-Za-z]|del)', mutation)
        if match:
            position = int(match.group(2))
            orf, offset = get_offset(subregion, position)
            trans_mut = translate_mutation(mutation, orf, offset)
            if trans_mut:
                translated.append(trans_mut)
        else:
            print(f"Invalid mutation format: {mutation}")
    return translated

# Example usage
rdrp_mutations = dfs["RNA-dependent RNA polymerase"]["Mutation"].tolist()
clpro_mutations = dfs["3C-like proteinase"]["Mutation"].tolist()

# Translate mutations
translated_rdrp = translate_mutations(rdrp_mutations, "RdRp")
translated_clpro = translate_mutations(clpro_mutations, "3CLpro")

# Output results
print("Translated RdRp mutations:")
print("\n".join(translated_rdrp))
print("\nTranslated 3CLpro mutations:")
print("\n".join(translated_clpro))

# Save to CSV
translated_rdrp_df = pd.DataFrame(translated_rdrp, columns=["Mutation"])
translated_clpro_df= pd.DataFrame(translated_clpro, columns=["Mutation"])
translated_rdrp_df.to_csv("translated_RdRp_mutations.csv", index=False)
translated_clpro_df.to_csv("translated_3CLpro_mutations.csv", index=False)
print("\nResults saved to CSV files.")

Translated RdRp mutations:
ORF1b:V157A
ORF1b:V157L
ORF1b:N189S
ORF1b:R276C
ORF1b:A367V
ORF1b:A440V
ORF1b:F471L
ORF1b:D475Y
ORF1b:A517V
ORF1b:V548L
ORF1b:G662S
ORF1b:S750A
ORF1b:V783I
ORF1b:E787G
ORF1b:C790F
ORF1b:C790R
ORF1b:E793A
ORF1b:E793D
ORF1b:M915R

Translated 3CLpro mutations:
ORF1a:T3284I
ORF1a:T3288A
ORF1a:T3288N
ORF1a:T3308I
ORF1a:D3311Y
ORF1a:M3312I
ORF1a:M3312L
ORF1a:M3312T
ORF1a:M3312d
ORF1a:L3313F
ORF1a:G3401S
ORF1a:F3403L
ORF1a:F3403S
ORF1a:N3405D
ORF1a:N3405L
ORF1a:N3405S
ORF1a:G3406S
ORF1a:S3407A
ORF1a:S3407E
ORF1a:S3407L
ORF1a:S3407P
ORF1a:C3423F
ORF1a:M3428R
ORF1a:M3428T
ORF1a:E3429A
ORF1a:E3429G
ORF1a:E3429K
ORF1a:E3429Q
ORF1a:E3429V
ORF1a:L3430F
ORF1a:P3431d
ORF1a:T3432I
ORF1a:H3435L
ORF1a:H3435N
ORF1a:H3435Q
ORF1a:H3435Y
ORF1a:A3436T
ORF1a:A3436V
ORF1a:V3449A
ORF1a:R3451G
ORF1a:R3451S
ORF1a:Q3452I
ORF1a:Q3452K
ORF1a:T3453I
ORF1a:A3454T
ORF1a:A3454V
ORF1a:Q3455A
ORF1a:Q3455C
ORF1a:Q3455D
ORF1a:Q3455E
ORF1a:Q3455F
ORF1a:Q3455G
ORF1a:Q3455H
ORF1a:Q3455I
ORF1a:Q3455K


In [2]:
import requests

# Fetch the reference genome from the API
url = "https://lapis.cov-spectrum.org/open/v2/sample/referenceGenome?downloadAsFile=false"
response = requests.get(url)
response.raise_for_status()
ref_json = response.json()

print("Reference genome fetched successfully.")

print(ref_json.keys())


Reference genome fetched successfully.
dict_keys(['nucleotideSequences', 'genes'])


In [3]:
# given the ref json - compar if at the correct position we have the same amino acid
ref_json["genes"]

[{'name': 'E',
  'sequence': 'MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*'},
 {'name': 'M',
  'sequence': 'MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*'},
 {'name': 'N',
  'sequence': 'MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*'},
 {'name': 'ORF1a',
  'sequence': 'MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDL

In [4]:
genes = {}
for gene in ref_json["genes"]:
    genes[gene["name"]] = gene

genes

{'E': {'name': 'E',
  'sequence': 'MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*'},
 'M': {'name': 'M',
  'sequence': 'MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*'},
 'N': {'name': 'N',
  'sequence': 'MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*'},
 'ORF1a': {'name': 'ORF1a',
  'sequence': 'MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKV

In [5]:
# not get the amino acid synbole at the position one-based
def get_aa_at_position(gene, position):
    """Get the amino acid at a specific position in a gene."""
    if position < 1 or position > len(gene["sequence"]):
        raise ValueError("Position out of range")
    return gene["sequence"][position - 1]  # Convert to zero-based index

# Example usage
gene_name = "ORF1a"  # Replace with the desired gene name
position = 1000
if gene_name in genes:
    gene = genes[gene_name]
    try:
        aa = get_aa_at_position(gene, position)
        print(f"Amino acid at position {position} in {gene_name}: {aa}")
    except ValueError as e:
        print(e)



Amino acid at position 1000 in ORF1a: T


In [13]:
# next check if rdrp_df and clpro_df did indeed have the same amino acid at the position

def check_mutation_consistency(mutations, gene):
    """Check if mutations are consistent with the reference gene."""
    for mutation in mutations:
        # split by :
        if ":" in mutation:
            mutation = mutation.split(":")[1]
        match = re.match(r'([A-Za-z])(\d+)([A-Za-z]|del)', mutation)
        if match:
            original = match.group(1)
            position = int(match.group(2))

            print(f"Checking mutation {mutation} at position {position} in gene {gene['name']}")
            new = match.group(3)
            try:
                aa_at_position = get_aa_at_position(gene, position)
                if aa_at_position != original:
                    print(f"Mutation {mutation} is inconsistent with reference at position {position}")
                    print(f"Expected {original}, found {aa_at_position}")
            except ValueError as e:
                print(e)
        else:
            print(f"Invalid mutation format: {mutation}")

# Check consistency for RdRp and 3CLpro
print("\nChecking consistency for RdRp mutations:")
check_mutation_consistency(translated_rdrp_df["Mutation"].to_list(), genes["ORF1b"])
print("\nChecking consistency for 3CLpro mutations:")
check_mutation_consistency(translated_clpro_df["Mutation"].to_list(), genes["ORF1a"])




Checking consistency for RdRp mutations:
Checking mutation V157A at position 157 in gene ORF1b
Checking mutation V157L at position 157 in gene ORF1b
Checking mutation N189S at position 189 in gene ORF1b
Checking mutation R276C at position 276 in gene ORF1b
Checking mutation A367V at position 367 in gene ORF1b
Checking mutation A440V at position 440 in gene ORF1b
Checking mutation F471L at position 471 in gene ORF1b
Checking mutation D475Y at position 475 in gene ORF1b
Checking mutation A517V at position 517 in gene ORF1b
Checking mutation V548L at position 548 in gene ORF1b
Checking mutation G662S at position 662 in gene ORF1b
Checking mutation S750A at position 750 in gene ORF1b
Checking mutation V783I at position 783 in gene ORF1b
Checking mutation E787G at position 787 in gene ORF1b
Checking mutation C790F at position 790 in gene ORF1b
Checking mutation C790R at position 790 in gene ORF1b
Checking mutation E793A at position 793 in gene ORF1b
Checking mutation E793D at position 793 

In [89]:
clpro_df["Mutation"].to_list()

['ORF1a:T3284I',
 'ORF1a:T3288A',
 'ORF1a:T3288N',
 'ORF1a:T3308I',
 'ORF1a:D3311Y',
 'ORF1a:M3312I',
 'ORF1a:M3312L',
 'ORF1a:M3312T',
 'ORF1a:M3312d',
 'ORF1a:L3313F',
 'ORF1a:G3401S',
 'ORF1a:F3403L',
 'ORF1a:F3403S',
 'ORF1a:N3405D',
 'ORF1a:N3405L',
 'ORF1a:N3405S',
 'ORF1a:G3406S',
 'ORF1a:S3407A',
 'ORF1a:S3407E',
 'ORF1a:S3407L',
 'ORF1a:S3407P',
 'ORF1a:C3423F',
 'ORF1a:M3428R',
 'ORF1a:M3428T',
 'ORF1a:E3429A',
 'ORF1a:E3429G',
 'ORF1a:E3429K',
 'ORF1a:E3429Q',
 'ORF1a:E3429V',
 'ORF1a:L3430F',
 'ORF1a:P3431d',
 'ORF1a:T3432I',
 'ORF1a:H3435L',
 'ORF1a:H3435N',
 'ORF1a:H3435Q',
 'ORF1a:H3435Y',
 'ORF1a:A3436T',
 'ORF1a:A3436V',
 'ORF1a:V3449A',
 'ORF1a:R3451G',
 'ORF1a:R3451S',
 'ORF1a:Q3452I',
 'ORF1a:Q3452K',
 'ORF1a:T3453I',
 'ORF1a:A3454T',
 'ORF1a:A3454V',
 'ORF1a:Q3455A',
 'ORF1a:Q3455C',
 'ORF1a:Q3455D',
 'ORF1a:Q3455E',
 'ORF1a:Q3455F',
 'ORF1a:Q3455G',
 'ORF1a:Q3455H',
 'ORF1a:Q3455I',
 'ORF1a:Q3455K',
 'ORF1a:Q3455L',
 'ORF1a:Q3455N',
 'ORF1a:Q3455P',
 'ORF1a:Q3455R